In [5]:
import requests
import json
import time
import sys
import re
import os
from requests.exceptions import RequestException
from bs4 import BeautifulSoup as bsoup
from contextlib import closing
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException

In [2]:
def simple_get(url,sess):
    try:
        with closing(sess.get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None
    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None

def is_good_response(resp):
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)

def log_error(e):
    print(e)

In [16]:
def write_file(data, filename, location='./college/'):
    if not os.path.exists(location):
        try:
            os.makedirs(location)
        except OSError as e:
            if e.errno != errno.EEXIST:
                raise
    path=os.path.join(location, filename + "." + 'json')
    with open(path,'w') as f:
        json.dump(data, f, sort_keys=True, indent=4)
    print('Done')
    return

In [8]:
def selenium_driver(url):
    prof=[]
    stud=[]
    time.sleep(10)
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument('--headless')
    driver = webdriver.Chrome("/usr/bin/chromedriver",options=options, service_log_path='./log/geckodriver.log')
    driver.get(url)
    ele=driver.find_elements(By.CSS_SELECTOR,"button.btn.btn-primary.btn-xs")
    for x in range(len(ele)):
        if ele[x].is_displayed():
            driver.execute_script("arguments[0].click();", ele[x])
            time.sleep(1)
    page_source = driver.page_source
    soup = bsoup(page_source, 'html.parser')
    table=soup.find("div",{"id":"showAlumniMembersDetails"})
    if table is not None:
        table_body = table.find('tbody')
        rows = table_body.find_all('tr')
        for h in rows[0].find('td').find_all(re.compile(r'h\d+')):
            prof.append(h.text.strip())
        rows.remove(rows[0])
        flag=True
        for row in rows:
            tmp=[]
            txt=row.find('td').text
            if 'Ph.D.' in str(txt):
                flag=True
            elif 'MS' in str(txt):
                flag=False
            else:
                if flag:
                    #print('phd')
                    tmp.append('phd')
                    for a in str(row).split("<br/>"):
                        b=re.sub(r"<[^>]*>", "", a)
                        tmp.append(b.strip())
                        #print(b)
                else:
                    #print('ms')
                    tmp.append('ms')
                    for a in str(row).split("<br/>"):
                        b=re.sub(r"<[^>]*>", "", a)
                        tmp.append(b.strip())
                        #print(b)
            if len(tmp) > 0:
                stud.append(tmp)
    else:
        return None
    driver.quit()
    return (prof, stud)

In [ ]:
options = webdriver.FirefoxOptions()
driver = webdriver.Firefox(options=options)

In [ ]:
dir(webdriver)

In [9]:
def iitkgp(url):
    base_url=url
    with requests.Session() as sess:
        cont=simple_get('http://www.iitkgp.ac.in/department/CS',sess)
        soup = bsoup(cont, 'html.parser')
        link=soup.find('div',attrs={'class':'aboutTheDepartmentFacultyListing'}).find_all('a')
        for l in link:
            new_url=base_url+l['href'].split(';')[0]+"#resp-tab4"
            yield new_url

In [6]:
#new_url='http://www.iitkgp.ac.in/department/CS/faculty/cs-abhij#resp-tab4'

In [ ]:
kgp={}
file=1
url='http://www.iitkgp.ac.in'
for nurl in iitkgp(url):
    try:
        prof_stud=selenium_driver(nurl)
        if prof_stud is not None:
            kgp[";".join(prof_stud[0])]=prof_stud[1]
    except WebDriverException as e:
        write_file(kgp,'iitkgp'+str(file))
        time.sleep(file*60)
        file+=1
        continue
write_file(kgp,'iitkgp'+str(file))

In [19]:
write_file(kgp,'iitkgp'+str(file))

Done


In [ ]:
'iitkgp'+str(file)

In [6]:
with open('./college/iitkgp1.json') as json_file:
    data = json.load(json_file)

In [8]:
conn=0
res=0
for a in data:
    res+=1+len(data[a])
    conn += len(data[a])

In [ ]:
for a in rows[0].find('td').find_all(re.compile(r'h\d+')):
    print(a.text.strip())

In [30]:
len(name1)

1300

In [31]:
a=set(name1)

In [32]:
len(a)

1083

In [35]:
import collections
a=[item for item, count in collections.Counter(name1).items() if count > 1]